# Using the HuggingSound library:

In [ ]:
!pip install torch==2.0.1 -qq

In [2]:
!pip install huggingsound -qq

      Successfully uninstalled torch-2.0.1+cu118
  Attempting uninstall: librosa
    Found existing installation: librosa 0.10.0.post2
    Uninstalling librosa-0.10.0.post2:
      Successfully uninstalled librosa-0.10.0.post2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.12.1 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 1.12.1 which is incompatible.
torchtext 0.15.2 requires torch==2.0.1, but you have torch 1.12.1 which is incompatible.
torchvision 0.15.2+cu118 requires torch==2.0.1, but you have torch 1.12.1 which is incompatible.


In [3]:
from huggingsound import SpeechRecognitionModel
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-russian", device=device)

INFO:huggingsound.speech_recognition.model:Loading model...


In [4]:
audio_paths = ["/content/demo_rus_tatsoft.wav", "/content/output.wav"]

In [5]:
transcriptions = model.transcribe(audio_paths)

100%|██████████| 2/2 [00:03<00:00,  1.55s/it]


In [6]:
for transcription in transcriptions:
    print(transcription['transcription'])

здравствуйте это возможно будущий голос машинного переводчика тотсофт
здравствуйте это пробный запуск это пбный запуск


# Writing your own inference script:

In [ ]:
!pip install datasets -qq
!pip install transformers -qq

In [8]:
import torch
import librosa
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

In [9]:
LANG_ID = "ru"
MODEL_ID = "jonatasgrosman/wav2vec2-large-xlsr-53-russian"
SAMPLES = 5

test_dataset = load_dataset("common_voice", LANG_ID, split=f"test[:{SAMPLES}]")

processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)

/root/.cache/huggingface/modules/datasets_modules/datasets/common_voice/220833898d6a60c50f621126e51fb22eb2dfe5244392c70dccd8e6e2f055f4bf/common_voice.py:634: FutureWarning: 
            This version of the Common Voice dataset is deprecated.
            You can download the latest one with
            >>> load_dataset("mozilla-foundation/common_voice_11_0", "en")
            
  warnings.warn(


Generating train split:   0%|          | 0/15481 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8007 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7963 [00:00<?, ? examples/s]

Generating other split:   0%|          | 0/10247 [00:00<?, ? examples/s]

Generating validated split:   0%|          | 0/74256 [00:00<?, ? examples/s]

Generating invalidated split:   0%|          | 0/3056 [00:00<?, ? examples/s]

In [10]:
# Preprocessing the datasets.
# We need to read the audio files as arrays
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = librosa.load(batch["path"], sr=16_000)
    batch["speech"] = speech_array
    batch["sentence"] = batch["sentence"].upper()
    return batch

In [11]:
test_dataset = test_dataset.map(speech_file_to_array_fn)
inputs = processor(test_dataset["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [ ]:
with torch.no_grad():
    logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

In [ ]:
predicted_ids = torch.argmax(logits, dim=-1)
predicted_sentences = processor.batch_decode(predicted_ids)

In [ ]:
for i, predicted_sentence in enumerate(predicted_sentences):
    print("-" * 100)
    print("Reference:", test_dataset[i]["sentence"])
    print("Prediction:", predicted_sentence)
